In [4]:
import tweepy
import json
import pandas as pd
import matplotlib.pyplot as plt
import collections
import seaborn as sns

# Añadir las claves (Yo las quito para evitar problemas con mi cuenta de desarrolador)
consumer_key = "n0zihT8RhgzbnoQoT02iLGFPp"
consumer_secret = "Jv72TVss1NBHsDoTezNdybuW1ors0M28u0IHCqCgLzIO8hNKDJ"
access_token = "1344609429543653377-9RC4xllEhCYOvYpKoF1dG4RO2sTVLT"
access_token_secret = "QwVL3WjYmV6yECUTBUBZkRbhsvMoDd0jJ6lrCjaQzhKWQ"

#Para OAuth 1.0 (Objeto con mis credenciales)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#Conexión a la API
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
datos = []        # Tabla a convertir en dataframe
num_tweets = 500  # Número de tweets a buscar

# Parámetros de la búsqueda que vamos ha realizar
tweetsBuscados = tweepy.Cursor(api.search,
                               q="ONG" or "inmigrante",
                               tweet_mode="extended",
                               lang="es",
                               since="2021-01-28",
                               until="2021-01-29",
                               excluded_replies = True,
                               include_rts = False).items(num_tweets)

for tweet in tweetsBuscados:
    datos.append([tweet.created_at, tweet.user.screen_name, tweet.full_text, tweet.retweet_count])

# Transformamos la tabla en un dataframe y lo mostramos por pantalla
datos = pd.DataFrame(datos, columns = ["Fecha", "Nombre_Usuario", "Tweet", "RTs"])
datos

In [2]:
# Creamos una lista que almacenará los usuarios
usuarios = []

# Recorremos el dataframe almacenando cada usuario en la lista
for i in datos.index:
    usuarios.append(datos["Nombre_Usuario"][i])
    
# Creamos una colección que cuente las veces que aperece el usuario en el dataframe, es decir, el número de
# veces que ha tuiteado sobre el tema
n_tweets = collections.Counter(usuarios).most_common(5)

# Guardamos cada dato de la colección en una lista separada para posteriormente graficar los resultados
listaUsers = []
listaTweets = []
for usuario, tweets in n_tweets:
    listaUsers.append(usuario)
    listaTweets.append(tweets)
print(listaUsers)
print(listaTweets)

fig, ax = plt.subplots()
# Ponemos una etiqueta al eje Y
ax.set_ylabel("Número de Tweets")
# Ponemos una etiqueta al eje X
ax.set_xlabel("Usuario")
# Ponemos los nombres de usuario en vertical para que quepan
plt.xticks(rotation="vertical")
# Sacamos la gráfica por pantalla
sns.barplot(x=listaUsers, y=listaTweets,  edgecolor="black", linewidth=1)
plt.savefig("usuariosTuiteros.jpg", bbox_inches="tight", dpi=200)

NameError: name 'datos' is not defined

In [5]:
class MiListener(tweepy.StreamListener):
    
    def __init__(self, api=None):
        #super().__init__()
        self.counter = 0
        self.limit = 10
    
    def on_connect(self):
        print("Conexión correcta!!!!")
             
    def on_data(self, data):
        
        status = json.loads(data)
        
        ###### Este bloque es para recuperar, correctamente, el texto del tweet #################################
        if 'text' in status:
            text = status['text']
        if 'extended_tweet' in status:
            text = status['extended_tweet']['full_text']
        if 'retweeted_status' in status:
            status_RT = status['retweeted_status']
            if 'text' in status_RT:
                text = 'RT @' + status['retweeted_status']['user']['screen_name'] + status_RT['text']    
                if 'extended_tweet' in status_RT:
                    extended_RT = status_RT['extended_tweet']
                    text = 'RT @' + status['retweeted_status']['user']['screen_name'] + extended_RT['full_text']
        ###### Este bloque es para recuperar, correctamente, el texto del tweet #################################
        
        
        ###### Para recuperar los hashtags que están dentro de una estructura de tipo diccionario #############
        hashtags = []
        for hashtag in status['entities']['hashtags']:
            hashtags.append(hashtag['text'])
        ###### Para recuperar los hashtags que están dentro de una estructura de tipo diccionario #############
        
        
        #### Este bloque sirve para almacenar alguna información del tweet en un fichero csv
        #### En este caso la fecha "status['created_at']",el screen_name del autor "status['user']['screen_name']",
        #### el texto del tweet "text.replace("\n", "")" y los hashtags "hashtags"
        #### Modificarlo para que también añada el nombre completo del autor y su localización
        # status["user"]["name"] --> Nombre completo del usuario
        # status["user"]["location"] --> Localización
        # añadimos %s por cada variable añadida al fichero y \t por cada separación de elementos
        with open("Tarea2.csv", "a", encoding='utf-8') as f:
            f.write("%s\t%s\t%s\t%s\t%s\t%s\n" % (status['created_at'],status['user']['screen_name'],status["user"]["name"],
                                              status["user"]["location"],text.replace("\n", ""), hashtags))
        
        self.counter += 1
        if self.counter < self.limit:
            return True
        else:
            return False

    def on_error(self, status_code):
        print("Error", status_code)

In [6]:
listener = MiListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True)) 
MiStreamer = tweepy.Stream(auth=auth, listener=listener)

MiStreamer.filter(track = ['immigrant'])

Conexión correcta!!!!


In [7]:
dfStream = pd.read_csv('Tarea2.csv', sep='\t', encoding = 'UTF-8', header = None)
dfStream.columns = ['Fecha','Autor','Nombre_Completo','Location','Tweet','Hashtags']
dfStream.shape
dfStream

,Fecha,Autor,Nombre_Completo,Location,Tweet,Hashtags
0,Sun Jan 31 15:02:52 +0000 2021,orgdoctor1,Barbara,"Kansas, USA",RT @stuartpstevensThe most dangerous immigrant...,[]
1,Sun Jan 31 15:02:52 +0000 2021,SarahPo77021908,Sarah,None,@catturd2 @Pamsta67 I think Biden is great 😊 I...,[]
2,Sun Jan 31 15:02:59 +0000 2021,MiamiPapers,Miami Secret Papers,"Miami, FL",RT @PnutBilly@HueyPNewton13 Which one is not l...,[]
3,Sun Jan 31 15:03:03 +0000 2021,DashOh1,D.D.,"Auburn, WA",@GaryGanjaGrows @thedgtalempress We can’t “tak...,[]
4,Sun Jan 31 15:03:06 +0000 2021,Akhgustavo,AbuAisha🇸🇴,None,@A__zak__ Yeah I believe they got Muhammad Kud...,[]
5,Sun Jan 31 15:03:06 +0000 2021,YNWA_Liverpool9,Bell,"Pennsylvania, USA",RT @jacobsoboroffTrump admin “separated far mo...,[]
6,Sun Jan 31 15:03:14 +0000 2021,john51652772,john_homeless_student,London,@TerryCretin @SimonJonesNews @ShadiaED @Shadia...,[]
7,Sun Jan 31 15:03:21 +0000 2021,NeejaB,Neeja Bakshi,None,Grieving during COVID times. When you are seco...,[]
8,Sun Jan 31 15:03:29 +0000 2021,KtbMoa,Ely Cheikh,None,RT @jacobinmagThe arrest of Russian opposition...,[]
9,Sun Jan 31 15:03:39 +0000 2021,freefilustine,Freedom and Justice Bee,None,RT @santafeezThe Enduring Harms of Trump’s Mus...,[]
